<a href="https://colab.research.google.com/github/vishalmysore/AI/blob/main/CookGPT_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
! pip install transformers

In [ ]:
model_name = "NumbersStation/nsql-350M"

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.51G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

In [ ]:
%load_ext sql

In [ ]:
%%sql
sqlite:///cookgpt.db

In [ ]:
%%sql
CREATE TABLE Recipe (
    recipe_id number,
    location text,
    name text,
    cook_time number,
    total_time number,
    calories number,
    people number,
    is_vegetarian others,
    is_vegan others,
    is_gluten_free others,
    is_dairy_free others,
    is_low_sugar others,
    is_very_healthy others,
    is_cheap others,
    is_very_popular others,
    is_sustainable others,
    instructions text,
    ingredients text,
    nutrients text,
    diet_labels text,
    health_labels text,
    cautions text,
    cuisine_type text,
    meal_type text,
    dish_type text,
    dish_type_others text
    )

 * sqlite:///cookgpt.db
Done.


[]

In [ ]:
%%sql
CREATE TABLE ratings (
    recipe_id number,
    rating number,
    user_id number,
    date text,
    comment text,
    useful number,
    funny number,
    cool number
)

 * sqlite:///cookgpt.db
Done.


[]

In [ ]:
%%sql
CREATE TABLE recipe_in_category (
    recipe_id number,
    category_id number
)

 * sqlite:///cookgpt.db
Done.


[]

In [ ]:
%%sql
CREATE TABLE category (
    category_id number,
    category_name text
)

 * sqlite:///cookgpt.db
Done.


[]

In [ ]:
%%sql
CREATE TABLE users (
    user_id number,
    user_name text
)

 * sqlite:///cookgpt.db
Done.


[]

In [ ]:
%%sql

INSERT INTO users (user_id, user_name) VALUES (1, 'Pink Floyd');
INSERT INTO users (user_id, user_name) VALUES (2, 'Deep Purple');
INSERT INTO users (user_id, user_name) VALUES (2, 'Vishal Mysore');

 * sqlite:///cookgpt.db
1 rows affected.
1 rows affected.
1 rows affected.


[]

In [ ]:
%%sql
SELECT * from users;

 * sqlite:///cookgpt.db
Done.


user_id,user_name
1,Pink Floyd
2,Deep Purple
2,Vishal Mysore


In [ ]:
table_schema = """CREATE TABLE Recipe (
    recipe_id number,
    location text,
    name text,
    cook_time number,
    total_time number,
    calories number,
    people number,
    is_vegetarian others,
    is_vegan others,
    is_gluten_free others,
    is_dairy_free others,
    is_low_sugar others,
    is_very_healthy others,
    is_cheap others,
    is_very_popular others,
    is_sustainable others,
    instructions text,
    ingredients text,
    nutrients text,
    diet_labels text,
    health_labels text,
    cautions text,
    cuisine_type text,
    meal_type text,
    dish_type text,
    dish_type_others text
    )

CREATE TABLE ratings (
    recipe_id number,
    rating number,
    user_id number,
    date text,
    comment text,
    useful number,
    funny number,
    cool number
)

CREATE TABLE recipe_in_category (
    recipe_id number,
    category_id number
)

CREATE TABLE category (
    category_id number,
    category_name text
)

CREATE TABLE users (
    user_id number,
    user_name text
)

CREATE TABLE recipe_in_cuisine (
    recipe_id number,
    cuisine_id number
)

CREATE TABLE cuisine (
    cuisine_id number,
    cuisine_name text
)
"""



In [ ]:
question = "give me recipe from south india ,I need to cook for 2 people and i can cook under 10 minutes? It should be 5 star rated recipe"

prompt = f"""{table_schema}

-- Using valid SQLite, answer the following questions for the tables provided above.

-- {question}

SELECT"""

In [ ]:
question = "give me recipe details with 5 star rating"

prompt = f"""{table_schema}

-- Using valid SQLite, answer the following questions for the tables provided above.

-- {question}

SELECT"""

In [ ]:
question = "give all the recipes ingredients for which user vishal has submitted ratings"

prompt = f"""{table_schema}

-- Using valid SQLite, answer the following questions for the tables provided above.

-- {question}

SELECT"""

In [ ]:
input_ids = tokenizer(prompt, return_tensors="pt").input_ids
generated_ids = model.generate(input_ids, max_length=500)
output = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
output = 'SELECT' + output.split('SELECT')[-1]

print(output)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  generation_config=generation_config, stopping_criteria=stopping_criteria


SELECT * FROM recipe WHERE people = 2 AND location = "South India" AND cook_time < 10;


In [ ]:
!wget https://tdmdal.github.io/mma-sql-2021/data/northwind.sqlite3

--2024-01-11 14:36:03--  https://tdmdal.github.io/mma-sql-2021/data/northwind.sqlite3
Resolving tdmdal.github.io (tdmdal.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to tdmdal.github.io (tdmdal.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 804864 (786K) [application/octet-stream]
Saving to: ‘northwind.sqlite3’

northwind.sqlite3   100%[===================>] 786.00K  5.02MB/s    in 0.2s    

2024-01-11 14:36:04 (5.02 MB/s) - ‘northwind.sqlite3’ saved [804864/804864]



In [ ]:
# load the SQL magic extension
# https://github.com/catherinedevlin/ipython-sql
# this extension allows us to connect to DBs and issue SQL command
%load_ext sql

# now we can use the magic extension to connect to our SQLite DB
# use %sql to write an inline SQL command
# use %%sql to write SQL commands in a cell
%sql sqlite:///northwind.sqlite3

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [ ]:
%%sql
SELECT sqlite_version();

 * sqlite:///northwind.sqlite3
Done.


sqlite_version()
3.37.2


In [ ]:
%%sql
SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite_%';

 * sqlite:///northwind.sqlite3
Done.


name
Suppliers
Shippers
Employees
Customers
Orders
Categories
Products
OrderDetails


In [ ]:
%%sql
SELECT * FROM sqlite_master

 * sqlite:///northwind.sqlite3
Done.


type,name,tbl_name,rootpage,sql
table,Suppliers,Suppliers,2,"CREATE TABLE [Suppliers] ( [SupplierID] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, [CompanyName] nvarchar(40) NOT NULL COLLATE NOCASE, [ContactName] nvarchar(30) NULL COLLATE NOCASE, [ContactTitle] nvarchar(30) NULL COLLATE NOCASE, [Address] nvarchar(60) NULL COLLATE NOCASE, [City] nvarchar(15) NULL COLLATE NOCASE, [Region] nvarchar(15) NULL COLLATE NOCASE, [PostalCode] nvarchar(10) NULL COLLATE NOCASE, [Country] nvarchar(15) NULL COLLATE NOCASE, [Phone] nvarchar(24) NULL COLLATE NOCASE, [Fax] nvarchar(24) NULL COLLATE NOCASE, [HomePage] ntext NULL)"
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,Shippers,Shippers,4,"CREATE TABLE [Shippers] ( [ShipperID] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, [CompanyName] nvarchar(40) NOT NULL COLLATE NOCASE, [Phone] nvarchar(24) NULL COLLATE NOCASE)"
table,Employees,Employees,5,"CREATE TABLE [Employees] ( [EmployeeID] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, [LastName] nvarchar(20) NOT NULL COLLATE NOCASE, [FirstName] nvarchar(10) NOT NULL COLLATE NOCASE, [Title] nvarchar(30) NULL COLLATE NOCASE, [TitleOfCourtesy] nvarchar(25) NULL COLLATE NOCASE, [BirthDate] datetime NULL, [HireDate] datetime NULL, [Address] nvarchar(60) NULL COLLATE NOCASE, [City] nvarchar(15) NULL COLLATE NOCASE, [Region] nvarchar(15) NULL COLLATE NOCASE, [PostalCode] nvarchar(10) NULL COLLATE NOCASE, [Country] nvarchar(15) NULL COLLATE NOCASE, [HomePhone] nvarchar(24) NULL COLLATE NOCASE, [Extension] nvarchar(4) NULL COLLATE NOCASE, [Photo] image NULL, [Notes] ntext NULL, [ReportsTo] int NULL, [PhotoPath] nvarchar(255) NULL COLLATE NOCASE, CONSTRAINT [FK_Employees_Employees] FOREIGN KEY ([ReportsTo]) REFERENCES [Employees] ([EmployeeID]) ON DELETE NO ACTION ON UPDATE NO ACTION)"
table,Customers,Customers,8,"CREATE TABLE [Customers] ( [CustomerID] nchar(5) NOT NULL, [CompanyName] nvarchar(40) NOT NULL COLLATE NOCASE, [ContactName] nvarchar(30) NULL COLLATE NOCASE, [ContactTitle] nvarchar(30) NULL COLLATE NOCASE, [Address] nvarchar(60) NULL COLLATE NOCASE, [City] nvarchar(15) NULL COLLATE NOCASE, [Region] nvarchar(15) NULL COLLATE NOCASE, [PostalCode] nvarchar(10) NULL COLLATE NOCASE, [Country] nvarchar(15) NULL COLLATE NOCASE, [Phone] nvarchar(24) NULL COLLATE NOCASE, [Fax] nvarchar(24) NULL COLLATE NOCASE, CONSTRAINT [PK_Customers] PRIMARY KEY ([CustomerID]))"
index,sqlite_autoindex_Customers_1,Customers,9,None
table,Orders,Orders,11,"CREATE TABLE [Orders] ( [OrderID] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, [CustomerID] nchar(5) NOT NULL COLLATE NOCASE, [EmployeeID] int NULL, [OrderDate] datetime NULL, [RequiredDate] datetime NULL, [ShippedDate] datetime NULL, [ShipVia] int NULL, [Freight] money DEFAULT 0 NULL, [ShipName] nvarchar(40) NULL COLLATE NOCASE, [ShipAddress] nvarchar(60) NULL COLLATE NOCASE, [ShipCity] nvarchar(15) NULL COLLATE NOCASE, [ShipRegion] nvarchar(15) NULL COLLATE NOCASE, [ShipPostalCode] nvarchar(10) NULL COLLATE NOCASE, [ShipCountry] nvarchar(15) NULL COLLATE NOCASE, CONSTRAINT [FK_Orders_Customers] FOREIGN KEY ([CustomerID]) REFERENCES [Customers] ([CustomerID]) ON DELETE NO ACTION ON UPDATE NO ACTION, CONSTRAINT [FK_Orders_Employees] FOREIGN KEY ([EmployeeID]) REFERENCES [Employees] ([EmployeeID]) ON DELETE NO ACTION ON UPDATE NO ACTION, CONSTRAINT [FK_Orders_Shippers] FOREIGN KEY ([ShipVia]) REFERENCES [Shippers] ([ShipperID]) ON DELETE NO ACTION ON UPDATE NO ACTION)"
table,Categories,Categories,13,"CREATE TABLE [Categories] ( [CategoryID] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, [CategoryName] nvarchar(15) NOT NULL COLLATE NOCASE, [Description] ntext NULL, [Picture] image NULL)"
table,Products,Products,14,"CREATE TABLE [Products] ( [ProductID] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, [ProductName] nvarchar(40) NOT NULL COLLATE NOCASE, [SupplierID] int NULL, [CategoryID] int NULL, [QuantityPerUnit] nvarchar(20) NULL COLLATE NOCASE, [UnitPrice] money DEFAULT 0 NULL, [UnitsInStoc

In [ ]:
%%sql
-- find out relationships between tables using SQL command
SELECT sql
FROM sqlite_master
WHERE name = "Orders"

 * sqlite:///northwind.sqlite3
Done.


sql
"CREATE TABLE [Orders] ( [OrderID] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, [CustomerID] nchar(5) NOT NULL COLLATE NOCASE, [EmployeeID] int NULL, [OrderDate] datetime NULL, [RequiredDate] datetime NULL, [ShippedDate] datetime NULL, [ShipVia] int NULL, [Freight] money DEFAULT 0 NULL, [ShipName] nvarchar(40) NULL COLLATE NOCASE, [ShipAddress] nvarchar(60) NULL COLLATE NOCASE, [ShipCity] nvarchar(15) NULL COLLATE NOCASE, [ShipRegion] nvarchar(15) NULL COLLATE NOCASE, [ShipPostalCode] nvarchar(10) NULL COLLATE NOCASE, [ShipCountry] nvarchar(15) NULL COLLATE NOCASE, CONSTRAINT [FK_Orders_Customers] FOREIGN KEY ([CustomerID]) REFERENCES [Customers] ([CustomerID]) ON DELETE NO ACTION ON UPDATE NO ACTION, CONSTRAINT [FK_Orders_Employees] FOREIGN KEY ([EmployeeID]) REFERENCES [Employees] ([EmployeeID]) ON DELETE NO ACTION ON UPDATE NO ACTION, CONSTRAINT [FK_Orders_Shippers] FOREIGN KEY ([ShipVia]) REFERENCES [Shippers] ([ShipperID]) ON DELETE NO ACTION ON UPDATE NO ACTION)"


In [ ]:
%%sql
-- take a look at the Employees table first
-- use LIMIT to limit the rows being displayed
SELECT FirstName, LastName, HireDate, Title, Country
FROM Employees
LIMIT 10

UsageError: Cell magic `%%sql` not found.


In [ ]:
!git clone https://github.com/vishalmysore/NSQL.git

Cloning into 'NSQL'...
remote: Enumerating objects: 42, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (22/22), done.
Receiving objects: 100% (42/42), 39.66 KiB | 9.91 MiB/s, done.
remote: Total 42 (delta 11), reused 7 (delta 7), pack-reused 13
Resolving deltas: 100% (14/14), done.


In [ ]:
!cd NSQL && pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.3/75.3 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 75.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 66.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 37.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.7/246.7 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.3/250.3 kB 27.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 78.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 77.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 10.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.3/60.3 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 69.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
DATABASE = "cookgpt.db"
TABLES = []

In [ ]:
import sys
sys.path.append("NSQL/examples/")
from db_connectors import SQLiteConnector
from prompt_formatters import RajkumarFormatter

# Get the connector and formatter
sqlite_connector = SQLiteConnector(
    database_path=DATABASE
)
sqlite_connector.connect()
tab = []
tab.extend(sqlite_connector.get_tables())
print(f"Loading tables: {tab}")

db_schema = [sqlite_connector.get_schema(table) for table in TABLES]
formatter = RajkumarFormatter(db_schema)

Loading tables: ['Recipe', 'category', 'ratings', 'recipe_in_category', 'users']


In [ ]:
!pip install manifest-ml\[all\]

In [ ]:
!nohup python -m manifest.api.app --model_type huggingface --model_generation_type text-generation  --model_name_or_path NumbersStation/nsql-350M  --device 0 > output.log &

nohup: redirecting stderr to stdout


In [ ]:
from manifest import Manifest

manifest_client = Manifest(client_name="huggingface", client_connection="http://127.0.0.1:5000")

def get_sql(instruction: str, max_tokens: int = 300) -> str:
    prompt = formatter.format_prompt(instruction)
    res = manifest_client.run(prompt, max_tokens=max_tokens)
    return formatter.format_model_output(res)

In [ ]:
sql = get_sql("who are all the users")
print(get_sql(sql))

SELECT * FROM Users;


In [ ]:
print(sqlite_connector.run_sql_as_df(sql))

   user_id      user_name
0        1     Pink Floyd
1        2    Deep Purple
2        2  Vishal Mysore


In [ ]:
sql = get_sql("give me all the receipe from south india locaiton which i can cook in less than 10 mins")
print(get_sql(sql))

SELECT * FROM receipe WHERE receipe.locaiton ='south india locaiton' AND receipe.cook_in_mins < 10;
